### GloVe Embeddings

In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
# Download GloVe Embeddings and Load GloVe Embeddings
glove_file_path = "D:/SIBMB/SEM 4/DL/WORD2VEC/glove.6B/glove.6B.300d.txt"
embeddings_index = {}
with open(glove_file_path, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [3]:
# Read the CSV file
df = pd.read_csv("D:/SIBMB/SEM 4/DL/PROJECT/Disaster Dataset/train.csv", encoding='latin-1')


In [4]:
# Tokenizing and padding the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(df['text'])
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')


In [5]:
# Creating an embedding matrix using GloVe
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [6]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, pd.get_dummies(df['target']).values, test_size=0.2, random_state=42)


In [7]:
# Building the ANN model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Flatten())
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=len(df['target'].unique()), activation='softmax'))


In [8]:
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
# Training the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/50


191/191 [==============================] - 3s 7ms/step - loss: 0.5278 - accuracy: 0.7438 - val_loss: 0.4759 - val_accuracy: 0.7912
Epoch 2/50
191/191 [==============================] - 1s 5ms/step - loss: 0.3715 - accuracy: 0.8373 - val_loss: 0.4865 - val_accuracy: 0.7932
Epoch 3/50
191/191 [==============================] - 1s 5ms/step - loss: 0.2971 - accuracy: 0.8813 - val_loss: 0.5435 - val_accuracy: 0.7820
Epoch 4/50
191/191 [==============================] - 1s 4ms/step - loss: 0.2389 - accuracy: 0.9036 - val_loss: 0.5617 - val_accuracy: 0.7840
Epoch 5/50
191/191 [==============================] - 1s 5ms/step - loss: 0.1929 - accuracy: 0.9322 - val_loss: 0.6129 - val_accuracy: 0.7728
Epoch 6/50
191/191 [==============================] - 1s 5ms/step - loss: 0.1612 - accuracy: 0.9478 - val_loss: 0.6701 - val_accuracy: 0.7636
Epoch 7/50
191/191 [==============================] - 1s 4ms/step - loss: 0.1322 - accuracy: 0.9611 - val_loss: 0.7353 - val_accuracy: 0.7649
Epoc

In [10]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
# Print the accuracy
print(f'Test Accuracy: {accuracy * 100:.2f}%')

48/48 [==============================] - 0s 3ms/step - loss: 1.3370 - accuracy: 0.7564
Test Accuracy: 75.64%


### FastText

In [11]:
from gensim.models import KeyedVectors
from smart_open import open

# Specify the path to the FastText .vec file
fasttext_file_path = "D:/SIBMB/SEM 4/DL/WORD2VEC/FastText/wiki-news-300d-1M.vec"

# Load FastText Embeddings using Gensim and smart_open
with open(fasttext_file_path, 'rb') as f:
    fasttext_model = KeyedVectors.load_word2vec_format(f, binary=False)

In [12]:
# Tokenizing and padding the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(df['text'])
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [13]:
# Creating an embedding matrix using FastText
EMBEDDING_DIM = 300  # Adjust based on the dimension of your FastText embeddings
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if word in fasttext_model:
        embedding_matrix[i] = fasttext_model[word]

In [14]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, pd.get_dummies(df['target']).values, test_size=0.2, random_state=42)


In [15]:
# Building the ANN model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Flatten())
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=len(df['target'].unique()), activation='softmax'))


In [16]:
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:
# Training the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
191/191 [==============================] - 2s 5ms/step - loss: 0.5721 - accuracy: 0.7062 - val_loss: 0.4832 - val_accuracy: 0.7892
Epoch 2/50
191/191 [==============================] - 1s 4ms/step - loss: 0.4066 - accuracy: 0.8302 - val_loss: 0.4658 - val_accuracy: 0.7905
Epoch 3/50
191/191 [==============================] - 1s 4ms/step - loss: 0.3246 - accuracy: 0.8703 - val_loss: 0.4750 - val_accuracy: 0.7879
Epoch 4/50
191/191 [==============================] - 1s 6ms/step - loss: 0.2732 - accuracy: 0.8941 - val_loss: 0.5008 - val_accuracy: 0.7800
Epoch 5/50
191/191 [==============================] - 1s 4ms/step - loss: 0.2345 - accuracy: 0.9118 - val_loss: 0.5267 - val_accuracy: 0.7846
Epoch 6/50
191/191 [==============================] - 1s 4ms/step - loss: 0.2032 - accuracy: 0.9268 - val_loss: 0.5867 - val_accuracy: 0.7636
Epoch 7/50
191/191 [==============================] - 1s 4ms/step - loss: 0.1822 - accuracy: 0.9356 - val_loss: 0.6053 - val_accuracy: 0.7689
Epoch 

In [18]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print(f'Test Accuracy: {accuracy * 100:.2f}%')

48/48 [==============================] - 0s 3ms/step - loss: 1.2827 - accuracy: 0.7420
Test Accuracy: 74.20%
